### Step 0: Open hcltech_dynamodb_table_setup.ipynb notebook and run the steps -this will create 2 DynamoDB tables 

### Step 1: Open agent1-docextraction.ipynb notebook and run the steps - this will create the agent which will read the document from S3 bucket and extracts raw text, table text and key value pair, also stores them in DynamoDB tables

### Step 2: Open agent2-doc_classification.ipynb notebook and run the steps - this will create the agent which will rad the document raw text, table text and key value pair from DynamoDB tables and classifies the document

### Step 3: Open agent3_doc_entity_extraction.ipynb notebook and run the steps - this will fetch the config file (consisting of entities to be extracted) from S3 and then extracts those entities using LLM

### Step 4: Open agentcore-orchestrator-deployer.ipynb notebook and run the steps - 
#### 1. this will wrap the agents created above as Tools, 
#### 2. it will also create an AgentCore Runtime 
#### 3. create an Agent and creates agentcore configuration
#### 4. change the IAM role by updating the ".bedrock_agentcore.yaml" in respective places
####         a. execution role - arn:aws:iam::040504913362:role/HCL-User-Role-Aiml-BedrockAgentCore
####         b. code build role - arn:aws:iam::040504913362:role/HCL-User-Role-Aiml-BedrockAgentCore-CodeBuild
#### 5. agentcore runtime creates a Docker container, deploys the agentcore agent and provides the Agent ARN
#### 6. Setting up Amazon Cognito user pool
#### 7. Now you can invoke the agentcore agent directly from AWS Console or using the invoke method.....

In [3]:
# !pip install --force-reinstall -U -r requirements.txt --quiet

In [19]:
%%writefile agent4-orchestrator-deployer.py
# Export complete multi-agent system to main.py

# from strands import Agent, tool
# from strands.models import BedrockModel
# from strands.agent.conversation_manager import SummarizingConversationManager

# from agent1_docextraction_agent import doc_extraction
# from agent2_docprocessing_agent import get_classification

# from bedrock_agentcore import BedrockAgentCoreApp

# # from utils import get_guardrail_id

# app = BedrockAgentCoreApp()
# agent = Agent()

# ORCHESTRATOR_PROMPT = """You are a comprehensive legal advisor orchestrator that coordinates between specialized legal agents to provide complete legal guidance. 

# Your specialized agents are:
# 1. **agent1_docextraction_agent**: Handles document extraction and saving into database 
# 2. **agent2_docprocessing_agent**: Handles document classification and saving into database

# Guidelines for using your agents:
# - Use **agent1_docextraction_agent** for questions about: document extraction, document raw text and table text extraction
# - Use **agent2_docprocessing_agent** for questions about: document classification
# - You can first use agent1_docextraction_agent agent and only after successful completion of this agent, you can proceed with other agents
# - Always provide a cohesive summary that combines insights from multiple agents when applicable
# - Maintain a helpful, professional tone and include appropriate disclaimers about financial advice

# When a user asks a question:
# 1. Determine which agent(s) are most appropriate
# 2. Call the relevant agent(s) with focused queries
# 3. Synthesize the responses into a coherent, comprehensive answer
# 4. Provide actionable next steps when possible"""

# # # # Add conversation management to maintain context
# # # conversation_manager = SummarizingConversationManager(
# # #     summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
# # #     preserve_recent_messages=5,  # Always keep 5 most recent messages
# # # )
# print("inside orchestrator agent")
# # Continue with previous configurations
# bedrock_model = BedrockModel(
#     model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
#     region_name="us-east-1",
#     # temperature=0.0,  # Deterministic responses for financial advice
#     # guardrail_id=get_guardrail_id(),
#     # guardrail_version="DRAFT",
#     # guardrail_trace="enabled",
# )


# # Wrap document extraction Agent as a Tool
# @tool
# def doc_extraction_tool(query: str) -> str:
#     """Handle investment analysis queries including stock research, portfolio creation, and performance comparisons."""
#     try:
#         print("inside orchestrator agent-> doc_extraction_tool")
#         response = doc_extraction()
#         return str(response)
#     except Exception as e:
#         print("inside orchestrator agent-> Exception", str(e))
#         return f"❌ doc_extraction error: {str(e)}"

# # Wrap document extraction Agent as a Tool
# @tool
# def get_classification_tool(query: str) -> str:
#     """Handle investment analysis queries including stock research, portfolio creation, and performance comparisons."""
#     try:
#         print("inside orchestrator agent-> get_classification_tool")
#         response = get_classification(query)
#         return str(response)
#     except Exception as e:
#         print("inside get_classification_tool agent-> Exception", str(e))
#         return f"❌ get_classification_tool error: {str(e)}"
        
# Export complete multi-agent system to main.py

from strands import Agent, tool
from strands.models import BedrockModel
from strands.agent.conversation_manager import SummarizingConversationManager

from agent1_docextraction_agent import doc_extraction
from agent2_docclassification_agent import get_classification
from agent3_doc_entity_extraction import extract_entities_based_on_config


# from utils import get_guardrail_id

ORCHESTRATOR_PROMPT = """You are a comprehensive legal advisor orchestrator that coordinates between specialized legal agents to provide complete legal guidance. 

Your specialized agents are:
1. **agent1_docextraction_agent**: Handles document extraction and saving into database 
2. **agent2_docclassification_agent**: Handles document classification and saving into database
2. **agent3_doc_entity_extraction**: Handles Entity Extraction based on config file and saving into database

Guidelines for using your agents:
- Use **agent1_docextraction_agent** for questions about: document extraction, document raw text and table text extraction
- Use **agent2_docclassification_agent** for questions about: document classification
- Use **agent3_doc_entity_extraction** for questions about: document entity extraction
- You can first use agent1_docextraction_agent agent and only after successful completion of this agent, you can proceed with agent2_docclassification_agent agents and only after successful completion of this agent, you can proceed with agent3_doc_entity_extraction agent
- Always provide a cohesive summary that combines insights from multiple agents when applicable
- Maintain a helpful, professional tone and include appropriate disclaimers about financial advice

When a user asks a question:
1. Determine which agent(s) are most appropriate
2. Call the relevant agent(s) with focused queries
3. Synthesize the responses into a coherent, comprehensive answer
4. Provide actionable next steps when possible"""

# # Add conversation management to maintain context
# conversation_manager = SummarizingConversationManager(
#     summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
#     preserve_recent_messages=5,  # Always keep 5 most recent messages
# )
print("inside orchestrator agent")
# Continue with previous configurations
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name="us-east-1",
    # temperature=0.0,  # Deterministic responses for financial advice
    # guardrail_id=get_guardrail_id(),
    # guardrail_version="DRAFT",
    # guardrail_trace="enabled",
)


# Wrap document extraction Agent as a Tool
@tool
def doc_extraction_tool(docid, indexid, s3files) -> str:
    """Handle investment analysis queries including stock research, portfolio creation, and performance comparisons."""
    try:
        print("inside orchestrator agent-> doc_extraction_tool")
        response = doc_extraction(docid, indexid, s3files)
        return str(response)
    except Exception as e:
        print("inside orchestrator agent-> Exception", str(e))
        return f"❌ doc_extraction error: {str(e)}"

# Wrap document extraction Agent as a Tool
@tool
def get_classification_tool(docid) -> str:
    """Handle investment analysis queries including stock research, portfolio creation, and performance comparisons."""
    try:
        print("inside orchestrator agent-> get_classification_tool")
        response = get_classification(docid)
        return str(response)
    except Exception as e:
        print("inside get_classification_tool agent-> Exception", str(e))
        return f"❌ get_classification_tool error: {str(e)}"

# Wrap document extraction Agent as a Tool
@tool
def get_doc_entity_extraction_tool(docid, classification, s3configfile) -> str:
   
    try:
        print("inside orchestrator agent-> get_doc_entity_extraction_tool")
        response = extract_entities_based_on_config(docid, classification, s3configfile)
        return str(response)
    except Exception as e:
        print("inside get_doc_entity_extraction_tool agent-> Exception", str(e))
        return f"❌ get_doc_entity_extraction_tool error: {str(e)}"

################################################################################################
from bedrock_agentcore import BedrockAgentCoreApp
app = BedrockAgentCoreApp()
agent = Agent()


orchestrator_agent = Agent(
    model=bedrock_model,
    system_prompt=ORCHESTRATOR_PROMPT,
    tools=[doc_extraction_tool, get_classification_tool,get_doc_entity_extraction_tool],
)

@app.entrypoint
async def invoke(payload):
    """Your AI agent function"""
    user_message = payload["prompt"]
    # user_message =  "1) Extract the document whose document id is DOC417927, index id is IN434221 and the s3 filepath is newmexicomutual/claimforms/IN434221/DOC417927/LegalCaseDocument-WC.pdf, 2) after extraction classify the document and then 3) extract the entities based on classification and s3 config file path newmexicomutual/yaju/legal_entity_extraction_config.txt "
    print("user sent the payload as --- ", user_message)
    async for event in orchestrator_agent.stream_async(user_message):
        if "data" in event:
            # Only stream text chunks to the client
            yield event["data"]


if __name__ == "__main__":
    app.run()

Overwriting agentcore-orchestrator-deployer.py


### To test the orchestrator agent locally follow belew steps 

In [17]:
# To test your Agent server locally:

# 1. **Open Terminal 1**: Start the Agent server
#    ```bash
#    python agentcore-orchestrator-deployer.py
#    ```
   
# 2. **Open Terminal 2**: Use the following command:
#    ```bash
#    curl -X POST http://localhost:8080/invocations \
#       -H "Content-Type: application/json" \
#       -d '{"prompt": "Extract the document whose document id is DOC417927 and classify the document"}'
#    ```

# You should see your agent response.

In [2]:
# !python agentcore-orchestrator-deployer.py 

In [21]:
# Import AgentCore Runtime deployment tools and utilities
import uuid
from utils import setup_cognito_user_pool, reauthenticate_user
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
from typing import Any, Optional
import urllib.parse
import requests
import json

In [3]:
# Configure AgentCore Runtime deployment settings
agentcore_runtime = Runtime()

agent_name = "hcltech_legal_orchestrator_agent"

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="agent4-orchestrator-deployer.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region="us-east-1",
    agent_name=agent_name,
    # authorizer_configuration=auth_config,
)

print("Configuration completed ✓")

Entrypoint parsed: file=/home/ec2-user/SageMaker/agentcore-samples/yaju-idp-agents/agentcore-orchestrator-deployer.py, bedrock_agentcore_name=agentcore-orchestrator-deployer
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: yaju_legal_orchestrator_agent1
Memory disabled
Network mode: PUBLIC


Configuring AgentCore Runtime...


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/ec2-user/SageMaker/agentcore-samples/yaju-idp-agents/.dockerignore
Keeping 'yaju_legal_orchestrator_agent1' as default agent
Bedrock AgentCore configured: /home/ec2-user/SageMaker/agentcore-samples/yaju-idp-agents/.bedrock_agentcore.yaml


Configuration completed ✓


### Change the IAM ROLE SO THAT BELOW STEP CAN RUN SMOOTHLY
##### run this commain and edit this hidden file "nano .bedrock_agentcore.yaml", give below IAM roles in respective places
##### execution role - arn:aws:iam::040504913362:role/HCL-User-Role-Aiml-BedrockAgentCore
##### code build role - arn:aws:iam::040504913362:role/HCL-User-Role-Aiml-BedrockAgentCore-CodeBuild


In [3]:
# # Deploy agent to AgentCore Runtime (creates ECR repo and runtime)
# print("Launching Agent server to AgentCore Runtime...")
# print("This may take several minutes...")

# launch_result = agentcore_runtime.launch(
#     env_vars={"OTEL_PYTHON_EXCLUDED_URLS": "/ping,/invocations"}
# )

# print("Launch completed ✓")
# print(f"Agent ARN: {launch_result.agent_arn}")
# print(f"Agent ID: {launch_result.agent_id}")

In [4]:
# # Set up Amazon Cognito for AgentCore Runtime authentication
# print("Setting up Amazon Cognito user pool...")

# cognito_config = setup_cognito_user_pool()

# print("Cognito setup completed ✓")
# print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
# print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

In [6]:
# # Configure JWT authorization for AgentCore Runtime
# auth_config = {
#     "customJWTAuthorizer": {
#         "allowedClients": [cognito_config["client_id"]],
#         "discoveryUrl": cognito_config["discovery_url"],
#     }
# }

In [5]:
# # Authenticate user and get bearer token for API access
# bearer_token = reauthenticate_user(client_id=cognito_config["client_id"])
# print(bearer_token)

In [25]:
# def invoke_endpoint(
#     agent_arn: str,
#     payload,
#     session_id: str,
#     bearer_token: Optional[str],
#     region: str = "us-east-1",
#     endpoint_name: str = "DEFAULT",
# ) -> Any:
#     """Invoke agent endpoint using HTTP request with bearer token."""
#     escaped_arn = urllib.parse.quote(agent_arn, safe="")
#     url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"
#     print("endpoint URL = ", url)
#     headers = {
#         "Authorization": f"Bearer {bearer_token}",
#         "Content-Type": "application/json",
#         "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
#     }

#     try:
#         body = json.loads(payload) if isinstance(payload, str) else payload
#     except json.JSONDecodeError:
#         body = {"payload": payload}
#     print("body  = ", body)
#     try:
#         response = requests.post(
#             url,
#             params={"qualifier": endpoint_name},
#             headers=headers,
#             json=body,
#             timeout=100,
#             stream=True,
#         )
#         print("------------------------ \n response  = ", response)
#         last_data = False
        
#         for line in response.iter_lines(chunk_size=1):
#             if line:
#                 line = line.decode("utf-8")
#                 if line.startswith("data: "):
#                     last_data = True
#                     line = line[6:]
#                     line = line.replace('"', "")
#                     yield line
#                 elif line:
#                     line = line.replace('"', "")
#                     if last_data:
#                         yield "\n" + line
#                     last_data = False

#     except requests.exceptions.RequestException as e:
#         print("Failed to invoke agent endpoint: %s", str(e))
#         raise

In [9]:
# print("launch_result.agent_arn = ", launch_result.agent_arn)

launch_result.agent_arn =  arn:aws:bedrock-agentcore:us-east-1:040504913362:runtime/yaju_legal_orchestrator_agent1-WSOnMB5tMk


### Below invokation will give error because public URL is restricted in HCLTech, so best is to test the same from AWS Console
### Please navigate to AWS Console - AgentCore Runtime - Agent Testing - provide the same prompt in curly braces liek below
####    {
####        "prompt": "1) Extract the document whose document id is DOC417927, index id is IN434221 and the s3 filepath is newmexicomutual/claimforms/IN434221/DOC417927/LegalCaseDocument-WC.pdf, 2) after extraction classify the document and then 3) extract the entities based on classification and s3 config file path newmexicomutual/yaju/legal_entity_extraction_config.txt "
####    }


In [1]:

# for chunk in invoke_endpoint(
#     agent_arn=launch_result.agent_arn,
#     payload={
#         "prompt": "1) Extract the document whose document id is DOC417927, index id is IN434221 and the s3 filepath is newmexicomutual/claimforms/IN434221/DOC417927/LegalCaseDocument-WC.pdf, 2) after extraction classify the document and then 3) extract the entities based on classification and s3 config file path newmexicomutual/yaju/legal_entity_extraction_config.txt "
#     },
#     session_id=str(uuid.uuid4()),
#     bearer_token=bearer_token,
# ):
#     print(chunk.replace("\\n", "\n"), end="")
#         # "prompt": "Extract the document whose document id is DOC417927 and classify the document "